# Summary - Web Scraping and APIs

In Python, if you think "There must be an easier way to do this!" then you are probably right.

Below is an example of custom code to pull finance data. it uses the Python requests library and regular expressions to parse the pages.

However, there is an API for this data through the [Pandas DataReader](https://pandas-datareader.readthedocs.io/en/latest/remote_data.html) or through Quandl.

I prefer Quandl to PDR. It is more stable and has additional functions. To use Quandl, you need to register for a free account at www.quandl.com.

# Tutorial Overview
This tutorial is divided into 3 part:
1. A custom web scraping solution (Not recommended)
2. Example with Pandas DataReader
3. Example with Quandl
4. You're turn!

In [ ]:
# https://flinhong.com/2017/10/30/query-yahoo-finance-historical-data-via-python-requests/

import re
import sys
import time
import datetime
import requests
import pandas as pd


def get_cookie_value(r):
    return {'B': r.cookies['B']}

def get_page_data(symbol):
    url = "https://finance.yahoo.com/quote/%s/?p=%s" % (symbol, symbol)
    r = requests.get(url)
    cookie = get_cookie_value(r)
    lines = r.content.decode('unicode-escape').strip(). replace('}', '\n')
    return cookie, lines.split('\n')

def find_crumb_store(lines):
    for l in lines:
        if re.findall(r'CrumbStore', l):
            return l
    print("Did not find CrumbStore")

def split_crumb_store(v):
    return v.split(':')[2].strip('"')

def get_cookie_crumb(symbol):
    cookie, lines = get_page_data(symbol)
    crumb = split_crumb_store(find_crumb_store(lines))
    return cookie, crumb

def get_data(symbol, start_date, end_date, cookie, crumb):
    filename = '%s.csv' % (symbol)
    url = "https://query1.finance.yahoo.com/v7/finance/download/%s?period1=%s&period2=%s&interval=1d&events=history&crumb=%s" % (symbol, start_date, end_date, crumb)
    response = requests.get(url, cookies=cookie)
    return response

def get_now_epoch():
    return int(time.time())

def convert_to_dataframe(data, symbol):
    data = data.content.decode("utf-8").splitlines()
    data = [i.strip(',') for i in data]
    data = pd.DataFrame(data)
    data = data[0].str.split(',', expand=True)
    cols = data.iloc[0,:].values.tolist()
    data.columns = cols
    data = data[1:]
    data['symbol'] = symbol
    data['Date'] = pd.DatetimeIndex(data['Date'])
    data[['Open','High','Low','Close','Adj Close','Volume']] = data[['Open','High','Low','Close','Adj Close','Volume']].astype(float)
    return data

def download_quotes(symbols):

    if type(symbols)!=list:
        symbols = [symbols]
        
    start_date = 0
    end_date = get_now_epoch()
    
    stocks = []
    for symbol in symbols:
        cookie, crumb = get_cookie_crumb(symbol)
        data = get_data(symbol, start_date, end_date, cookie, crumb)
        stocks.append(convert_to_dataframe(data, symbol))
        
    return pd.concat(stocks, axis=0)


In [ ]:
df = download_quotes(['AMZN'])
df.head()

# Example with Pandas DataReader

In [ ]:
import pandas_datareader.data as web
from config.quandl_config import QUANDL_API_KEY as api_key # file with my API key

symbol = 'WIKI/AMZN'

df = web.DataReader(symbol, 'quandl', api_key = api_key)
df.tail()

# Example with Quandl

In [ ]:
import quandl
from config.quandl_config import QUANDL_API_KEY as api_key # file with my API key

df = quandl.get("WIKI/AMZN", trim_start="1975-01-01", authtoken=api_key)
df.tail()

In [ ]:
df.loc['2012-02-15']['Close']

# Your Turn!


1\. What was the "Open" price of Amazon stock (AMZN) on February 15, 2012?

2\. What was the highest volume of Amazon stock traded in one day?


3\. On what day day did this occur?

4\. What was Amazon's biggest one day stock growth in dollar terms? in percentage terms?

5\. Create line chart showing Amazon stock price over time

6\. Create line chart showing average monthly stock price of Amazon (AMZN) vs S&P 500 (^GSPC) since January 2000 using Adj Close

hint: df['column'].dt.to_period("M") can convert a date to a month

7\. Create same chart in 2) but normalize it so that both stocks are set to 1.0 in January 2000

# Extensions

This section lists some ideas for extending the tutorial that you may wish to explore.
* Describe three examples when using APIs for data would be better than downloading data as spreadsheets.

# Further Reading
This section provides more resources on the topic if you are looking to go deeper.

## Books
* Python for Finance, by Yves Hilpisch. http://shop.oreilly.com/product/0636920032441.do
 * [Git Repo](https://github.com/yhilpisch/py4fi2nd)
* Mastering Python for Finance,  James Ma Weiming. https://www.amazon.com/dp/1789346460/
 * [Git Repo](https://github.com/PacktPublishing/Mastering-Python-for-Finance-Second-Edition)
* Python Finance Cookbook, Eryk Lewinson. https://www.packtpub.com/data/python-for-finance-cookbook
 * [Git Repo](https://github.com/erykml/Python-for-Finance-Cookbook)

## APIs
* Pandas DataReader. https://pandas-datareader.readthedocs.io/en/latest/ 
* Quandl. https://www.quandl.com/tools/python
    
# Summary

In this tutorial, you were introduced to pulling data using web scraping, Pandas DataReader, and Quandl. Specifically, you learned:
* How to use Pandas DataReader and Quandl to access stock data as a Pandas DataFrame.

# Next

Use the easiest option to get data and then take the data and do amazing analysis.